In [1]:
from bs4 import BeautifulSoup
import re
import requests
import operator
import threading
from threading import Thread
import pickle

In [2]:
file_base = %pwd
file_base += "/cve-data/"
file_base

'/Users/bala/Desktop/227/code/CSE-227/cve-data/'

In [3]:
CVE_IDS = []
file = "download ("
for i in range(0,29):
    file_path = file_base + file + str(i) + ").tsv"
    with open(file_path) as f:
        contents = f.readlines()
        for line in contents[1:]:
            words = line.split('\t')
            CVE_IDS.append(words[1])

In [11]:
CVE_TO_BUGID = dict()

for cve in CVE_IDS:
    url = "http://www.cvedetails.com/cve/" + cve + "/"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    
    table = soup.find("table", {"id": "vulnrefstable"})
    a_tags = table.find_all('a')
    for a in a_tags:
        if "bugzilla" in a['href']:
            if cve in CVE_TO_BUGID:
                CVE_TO_BUGID[cve].append(a['href'])
            else:
                CVE_TO_BUGID[cve] = [a['href']]

# some CVE entries don't have bugzilla links
print(len(CVE_TO_BUGID))

1233


In [14]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [16]:
#storing the dict
save_obj(CVE_TO_BUGID, "cve_to_bugzilla_links")

In [ ]:
for key_cve, value_links in CVE_TO_BUGID.items():
    for l in value_links:
        s_url = l
        s_hdr = {'User-Agent': 'Mozilla/5.0'}
        s_html = requests.get(s_url)
        s_soup = BeautifulSoup(s_html.text, "html.parser")
        